## 简答题
1. 如果已经在完全相同的训练集上训练了5个不同的模型，并且它们都达到了95%的准确率，是否还有机会获得更好的效果？如果可以，该怎么做？ 如果不行，为什么？可以，用模型结果投票得到新预测
2. 硬投票分类器和软投票分类器有什么区别？硬投票得到0/1...类别，软投票得到概率
3. 是否可以通过在多个服务器上并行来加速bagging集成的训练？pasting集成呢？提升集成呢？随机森林或堆叠集成呢？
可以并行来加速bagging/pasting集成。提升集成/随机森林基于上一次的模型无法并行，堆叠模型训练可以并行
4. 包外评估的好处是什么？
自带分离测试集
5. 是什么让极端随机树比一般随机森林更加随机？这部分增加的随机性有什么用？极端随机树比一般随机森林快还是慢？
随机分裂方差更低，提升泛化能力。训练更快
6. 如果AdaBoost集成对训练数据欠拟合，应该调整哪些超参数？怎么调整？
增加估计器，增大学习率，减少正则化
7. 如果梯度提升集成对训练集过拟合，应该提升还是降低学习率
降低学习率，早停

## 编程题
1. 加载MNIST数据集（**8_sklearn做分类** 里有介绍），将其分为一个训练集、一个验证集和一个测试集（例如使用40000个实例训练，10000个实例验证，最后20000个实例测试）。然后训练多个分类器，比如一个随机森林分类器、一个极端随机树分类器和一个SVM。接下来，尝试使用软投票法或者硬投票法将它们组合成一个集成，这个集成在验证集上的表现要胜过它们各自单独的表现。成功找到集成后，在测试集上测试。与单个的分类器相比，它的性能要好多少？


2. 运行上一题中的单个分类器，用验证集进行预测，然后用预测结果创建一个新的训练集：新训练集中的每个实例都是一个向量，这个向量包含所有分类器对于一个图像的一组预测，目标值是图像的类。恭喜，你成功训练了一个混合器，结合第一层的分类器，它们一起构成了一个堆叠集成。现在在测试集上评估这个集成。对于测试集中的每个图像，使用所有的分类器进行预测，然后将预测结果提供给混合器，得到集成的预测。与前面训练的投票分类器相比，这个集成的结果如何？现在再次尝试使用StackingClassifier。你得到了更好的性能吗？如果是这样，为什么？

In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, parser="auto")

In [2]:
X, y = mnist.data, mnist.target
X_train, X_valid, X_test, y_train, y_valid, y_test = X[:40000], X[40000:50000],X[50000:], y[:40000], y[40000:50000], y[50000:]

In [3]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.svm import LinearSVC

voting_classifier = VotingClassifier(
    estimators = [("rf", RandomForestClassifier(n_estimators=300, max_leaf_nodes=32, random_state=42)),
                  ("et", ExtraTreesClassifier(n_estimators=300, max_leaf_nodes=32, random_state=43)),
                  ("svc", LinearSVC(random_state=42, C=0.05, dual=True)),
], voting = 'hard', n_jobs = -1)

voting_classifier.fit(X_train, y_train)
for name, clf in voting_classifier.named_estimators_.items():
    print(name, " valid accuracy =", clf.score(X_valid, y_valid.astype(int)), "train accuracy =", clf.score(X_train, y_train.astype(int)), "test accuracy =", clf.score(X_test, y_test.astype(int)))

print(voting_classifier.score(X_valid,y_valid))
print(voting_classifier.score(X_test,y_test))

rf  valid accuracy = 0.8683 train accuracy = 0.870175 test accuracy = 0.88015
et  valid accuracy = 0.8524 train accuracy = 0.8564 test accuracy = 0.86625
svc  valid accuracy = 0.8488 train accuracy = 0.876725 test accuracy = 0.8702
0.8718
0.8849


In [ ]:
import numpy as np
from sklearn.ensemble import StackingClassifier

X_valid_predict = np.empty((len(X_valid), len(voting_classifier.estimators_)))
for i, name_clf in enumerate(voting_classifier.named_estimators_.items()):
    _, clf = name_clf
    X_valid_predict[:, i] = clf.predict(X_valid)

final_estimator = RandomForestClassifier(random_state=43)
final_estimator.fit(X_valid_predict, y_valid)
X_test_predict = np.empty((len(X_test), len(voting_classifier.estimators_)))

for i, name_clf in enumerate(voting_classifier.named_estimators_.items()):
    _, clf = name_clf
    X_test_predict[:, i] = clf.predict(X_test)

final_estimator.score(X_test_predict, y_test)
X_train_full, y_train_full = np.vstack((X_train, X_valid)), np.concatenate((y_train, y_valid))

stacking_clf = StackingClassifier(
     estimators = [("rf", RandomForestClassifier(n_estimators=300, max_leaf_nodes=32, random_state=42)),
                  ("et", ExtraTreesClassifier(n_estimators=300, max_leaf_nodes=32, random_state=43)),
                  ("svc", LinearSVC(random_state=42, C=0.05, dual=True)),],
    final_estimator = RandomForestClassifier(random_state=43),
    cv = 5
)

stacking_clf.fit(X_train_full, y_train_full)
stacking_clf.score(X_test, y_test)